# Quantum Counting Algorithm

This notebook implements the Quantum Counting Algorithm using the Amazon Braket SDK.

The Quantum Counting Algorithm (Brassard, Hoyer, Mosca, Tapp 1998) estimates the number of solutions $M$ to a search problem without needing to find them. Given an oracle $O_f$ that marks $M$ solutions among $N = 2^n$ possible inputs, the algorithm combines Grover's oracle with Quantum Phase Estimation (QPE) to estimate $M$.

Knowing $M$ is useful on its own (e.g., estimating the size of a database satisfying certain criteria) and also enables computing the optimal number of Grover iterations $r_{\text{opt}} = \lfloor \frac{\pi}{4} \sqrt{N/M} \rceil$ for a subsequent search.

## Algorithm Overview

The Grover operator $G = D \cdot O_f$ acts on $n$ search qubits, where:
- $O_f = I - 2\sum_{x: f(x)=1} |x\rangle\langle x|$ is the oracle that phase-flips solution states
- $D = 2|s\rangle\langle s| - I$ is the diffusion operator with $|s\rangle = H^{\otimes n}|0\rangle^{\otimes n}$

In the two-dimensional subspace spanned by the solution states $|w\rangle$ and non-solution states $|w^\perp\rangle$, $G$ acts as a rotation by angle $2\theta$ where $\sin^2(\theta) = M/N$.

The eigenvalues of $G$ are $e^{\pm 2i\theta}$. Quantum Phase Estimation with $t$ counting qubits estimates $\theta$, from which we compute:

$$M = N \sin^2(\theta)$$

The circuit consists of:
1. Hadamard gates on all $t$ counting and $n$ search qubits
2. Controlled-$G^{2^k}$ for each counting qubit $k$
3. Inverse QFT on the counting register
4. Measurement of the counting register

## References

[[1] Brassard, Hoyer, Mosca, Tapp - Quantum Amplitude Amplification and Estimation (2000)](https://arxiv.org/abs/quant-ph/0005055)

[[2] Nielsen & Chuang - Quantum Computation and Quantum Information, Section 5.4.2](https://www.cambridge.org/highereducation/books/quantum-computation-and-quantum-information/01E10196D0A682A6AEFFEA52D53BE9AE)

In [ ]:
import math

import matplotlib.pyplot as plt
from braket.circuits import Circuit
from braket.devices import LocalSimulator

from braket.experimental.algorithms.grovers_search import build_oracle, grovers_search
from braket.experimental.algorithms.quantum_counting import (
    get_quantum_counting_results,
    quantum_counting_circuit,
    run_quantum_counting,
)

%matplotlib inline

## Example 1: Counting a single solution

We construct an oracle that marks the state $|101\rangle$ among all $2^3 = 8$ possible 3-qubit states. The true number of solutions is $M = 1$.

In [ ]:
# Build oracle for solution '101'
solution = "101"
oracle = build_oracle(solution)
n_search = len(solution)
n_counting = 4

# Build and display the quantum counting circuit
circ = quantum_counting_circuit(oracle, n_search, n_counting)
print(f"Qubits: {n_counting} counting + {n_search} search = {circ.qubit_count} total")
print(f"Circuit depth: {circ.depth}")

In [ ]:
# Run on local simulator
device = LocalSimulator()
task = run_quantum_counting(circ, device, shots=2000)
results = get_quantum_counting_results(task, n_search, n_counting)

print(f"Estimated number of solutions M = {results['M_estimate']:.3f}")
print(f"True number of solutions M = 1")
print(f"Estimated theta = {results['theta_estimate']:.4f}")
print(f"Total search space N = {results['N_total']}")

In [ ]:
# Visualize counting register measurement distribution
counts = results["measurement_counts"]
sorted_counts = dict(sorted(counts.items()))

plt.figure(figsize=(10, 4))
plt.bar(sorted_counts.keys(), sorted_counts.values())
plt.xlabel("Counting register outcome")
plt.ylabel("Counts")
plt.title(f"Quantum Counting: oracle '{solution}', M_est = {results['M_estimate']:.2f}")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

The measurement distribution shows two dominant peaks. These correspond to the two eigenvalues $e^{+2i\theta}$ and $e^{-2i\theta}$ of the Grover operator. Both peaks yield the same estimate of $M$.

## Example 2: Counting with a 2-qubit search space

A simpler example: the oracle marks $|11\rangle$ among $N = 4$ states. True $M = 1$.

In [ ]:
oracle_2q = build_oracle("11")
circ_2q = quantum_counting_circuit(oracle_2q, n_search_qubits=2, n_counting_qubits=4)

task_2q = run_quantum_counting(circ_2q, LocalSimulator(), shots=2000)
results_2q = get_quantum_counting_results(task_2q, n_search_qubits=2, n_counting_qubits=4)

print(f"Estimated M = {results_2q['M_estimate']:.3f} (true M = 1)")
print(f"Search space N = {results_2q['N_total']}")

## Precision analysis

More counting qubits yield higher precision. Below we sweep from 2 to 6 counting qubits and plot the estimation error.

In [ ]:
true_M = 1
sweep_solution = "11"
oracle_sweep = build_oracle(sweep_solution)
n_search_sweep = len(sweep_solution)

counting_range = range(2, 7)
errors = []

for t in counting_range:
    circ_t = quantum_counting_circuit(oracle_sweep, n_search_sweep, t)
    task_t = run_quantum_counting(circ_t, LocalSimulator(), shots=2000)
    res_t = get_quantum_counting_results(task_t, n_search_sweep, t)
    error = abs(res_t["M_estimate"] - true_M)
    errors.append(error)
    print(f"t={t}: M_est={res_t['M_estimate']:.4f}, error={error:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(list(counting_range), errors, "o-")
plt.xlabel("Number of counting qubits (t)")
plt.ylabel("|M_estimate - M_true|")
plt.title("Estimation error vs counting qubits")
plt.yscale("log")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Connection to Grover's Search

A key application of quantum counting is computing the optimal number of Grover iterations. Given the estimate $\hat{M}$, the optimal number of iterations is:

$$r_{\text{opt}} = \left\lfloor \frac{\pi}{4} \sqrt{\frac{N}{\hat{M}}} \right\rceil$$

Below we use the counting estimate to run Grover's search with the optimal iteration count.

In [ ]:
# Use the counting result from Example 1
M_est = results["M_estimate"]
N_total = results["N_total"]

# Compute optimal Grover iterations
if M_est > 0 and M_est < N_total:
    r_opt = round(math.pi / 4 * math.sqrt(N_total / M_est))
else:
    r_opt = 1

print(f"Estimated M = {M_est:.3f}")
print(f"Optimal Grover iterations: r_opt = {r_opt}")

# Run Grover's search with optimal iterations
oracle_grover = build_oracle(solution)
grover_circ = grovers_search(oracle_grover, n_qubits=n_search, n_reps=r_opt)
task_grover = LocalSimulator().run(grover_circ, shots=1000)
result_grover = task_grover.result()

# Show results on data qubits only (ignore ancilla)
print(f"\nGrover's search results (top 3):")
data_counts = {}
for bs, count in result_grover.measurement_counts.items():
    data_bits = bs[:n_search]
    data_counts[data_bits] = data_counts.get(data_bits, 0) + count
total_shots = sum(data_counts.values())
for bs, count in sorted(data_counts.items(), key=lambda x: -x[1])[:3]:
    print(f"  |{bs}> : {count/total_shots:.4f}")
print(f"\nTarget solution: |{solution}>")

## Run on a Managed Simulator (optional)

The cell below runs the quantum counting circuit on the Amazon Braket SV1 managed simulator.
This section is commented out by default. Uncomment to run on AWS.

In [ ]:
# from braket.aws import AwsDevice
# from braket.tracking import Tracker
#
# tracker = Tracker().start()
#
# managed_device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
# oracle_cloud = build_oracle("101")
# circ_cloud = quantum_counting_circuit(oracle_cloud, n_search_qubits=3, n_counting_qubits=4)
# task_cloud = run_quantum_counting(circ_cloud, managed_device, shots=1000)
# results_cloud = get_quantum_counting_results(task_cloud, n_search_qubits=3, n_counting_qubits=4)
#
# print(f"Estimated M = {results_cloud['M_estimate']:.3f}")
# print(f"N = {results_cloud['N_total']}")
#
# print(f"\nTask Summary")
# print(f"{tracker.quantum_tasks_statistics()}")
# print(f"Estimated cost: ${tracker.simulator_tasks_cost():.2f} USD")

Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).